In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 6
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [256, 128]  # 双层LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = (len(x) - 1) // batchSize

        for i in range(numBatches - 1):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 构建模型
class BiLSTM(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)

            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm, self.embeddedWords, dtype=tf.float32)
        
        # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2]
        concatedOutput = tf.concat(outputs, 2)
        
        # 去除最后时间步的输出作为全连接的输入
        finalOutput = concatedOutput[:, -1, :]
        
        outputSize = config.model.hiddenSizes[-1] * 2  # 因为是双向LSTM，最终的输出值是fw和bw的拼接，因此要乘以2
        output = tf.reshape(finalOutput, [-1, outputSize])  # reshape成全连接层的输入维度
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTM(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0

2018-12-28T11:10:25.104321, step: 36, loss: 0.6969190239906311, acc: 0.5312, auc: 0.5242, precision: 0.75, recall: 0.0484
2018-12-28T11:10:25.670206, step: 37, loss: 0.7040481567382812, acc: 0.5469, auc: 0.4711, precision: 0.75, recall: 0.05
2018-12-28T11:10:26.245076, step: 38, loss: 0.6903712153434753, acc: 0.5234, auc: 0.5187, precision: 0.75, recall: 0.0476
2018-12-28T11:10:26.826581, step: 39, loss: 0.6781790256500244, acc: 0.5312, auc: 0.6044, precision: 1.0, recall: 0.0164
2018-12-28T11:10:27.362435, step: 40, loss: 0.7109823226928711, acc: 0.4219, auc: 0.4613, precision: 0.2857, recall: 0.0282
2018-12-28T11:10:27.957127, step: 41, loss: 0.7060298919677734, acc: 0.4922, auc: 0.4846, precision: 0.8, recall: 0.0588
2018-12-28T11:10:28.508838, step: 42, loss: 0.6989051103591919, acc: 0.4609, auc: 0.5196, precision: 0.0, recall: 0.0
2018-12-28T11:10:29.096959, step: 43, loss: 0.6895378828048706, acc: 0.4219, auc: 0.5246, precision: 0.6667, recall: 0.0779
2018-12-28T11:10:29.736931, 

2018-12-28T11:11:27.337614, step: 104, loss: 0.6819145679473877, acc: 0.5469, auc: 0.5796, precision: 0.0, recall: 0.0
2018-12-28T11:11:27.919320, step: 105, loss: 0.690883994102478, acc: 0.5859, auc: 0.5215, precision: 0.0, recall: 0.0
2018-12-28T11:11:28.537239, step: 106, loss: 0.6984807252883911, acc: 0.4531, auc: 0.5096, precision: 0.0, recall: 0.0
2018-12-28T11:11:29.167202, step: 107, loss: 0.701971173286438, acc: 0.4844, auc: 0.4933, precision: 0.0, recall: 0.0
2018-12-28T11:11:29.765474, step: 108, loss: 0.7002499103546143, acc: 0.4688, auc: 0.4804, precision: 0.0, recall: 0.0
2018-12-28T11:11:30.313088, step: 109, loss: 0.6882403492927551, acc: 0.5391, auc: 0.552, precision: 1.0, recall: 0.0167
2018-12-28T11:11:30.870182, step: 110, loss: 0.692074179649353, acc: 0.4922, auc: 0.5521, precision: 0.5, recall: 0.0154
2018-12-28T11:11:31.480340, step: 111, loss: 0.6981812715530396, acc: 0.4688, auc: 0.4713, precision: 0.0, recall: 0.0
2018-12-28T11:11:32.058076, step: 112, loss: 0

2018-12-28T11:12:08.478171, step: 173, loss: 0.7027219533920288, acc: 0.4766, auc: 0.4509, precision: 0.0, recall: 0.0
2018-12-28T11:12:09.022932, step: 174, loss: 0.7030916810035706, acc: 0.4531, auc: 0.4589, precision: 0.0, recall: 0.0
2018-12-28T11:12:09.632352, step: 175, loss: 0.6936715841293335, acc: 0.4453, auc: 0.551, precision: 0.0, recall: 0.0
2018-12-28T11:12:10.176251, step: 176, loss: 0.7002276182174683, acc: 0.4844, auc: 0.467, precision: 0.0, recall: 0.0
2018-12-28T11:12:10.753884, step: 177, loss: 0.6916866898536682, acc: 0.5156, auc: 0.5389, precision: 0.0, recall: 0.0
2018-12-28T11:12:11.362739, step: 178, loss: 0.6936594247817993, acc: 0.4844, auc: 0.5196, precision: 0.0, recall: 0.0
2018-12-28T11:12:11.923454, step: 179, loss: 0.6951043605804443, acc: 0.5391, auc: 0.5097, precision: 0.0, recall: 0.0
2018-12-28T11:12:12.529206, step: 180, loss: 0.6920321583747864, acc: 0.5312, auc: 0.5203, precision: 0.0, recall: 0.0
2018-12-28T11:12:13.118837, step: 181, loss: 0.686

2018-12-28T11:13:10.197806, step: 241, loss: 0.6856728792190552, acc: 0.5469, auc: 0.5665, precision: 0.0, recall: 0.0
2018-12-28T11:13:10.844082, step: 242, loss: 0.6858155727386475, acc: 0.5, auc: 0.5745, precision: 0.0, recall: 0.0
2018-12-28T11:13:11.465172, step: 243, loss: 0.6908243298530579, acc: 0.5078, auc: 0.5404, precision: 0.0, recall: 0.0
2018-12-28T11:13:12.149719, step: 244, loss: 0.6916531920433044, acc: 0.5234, auc: 0.5185, precision: 0.0, recall: 0.0
2018-12-28T11:13:12.726067, step: 245, loss: 0.688926100730896, acc: 0.5078, auc: 0.5558, precision: 0.0, recall: 0.0
2018-12-28T11:13:13.276367, step: 246, loss: 0.6914622783660889, acc: 0.5, auc: 0.5229, precision: 0.0, recall: 0.0
2018-12-28T11:13:13.860525, step: 247, loss: 0.6861144304275513, acc: 0.5234, auc: 0.5823, precision: 0.0, recall: 0.0
2018-12-28T11:13:14.436833, step: 248, loss: 0.7078530788421631, acc: 0.4453, auc: 0.4692, precision: 0.0, recall: 0.0
2018-12-28T11:13:15.009585, step: 249, loss: 0.68864488

...

2018-12-28T11:18:38.329953, step: 638, loss: 0.5151090025901794, acc: 0.6094, auc: 0.8921, precision: 1.0, recall: 0.2188
2018-12-28T11:18:38.890851, step: 639, loss: 0.6282573938369751, acc: 0.5391, auc: 0.7367, precision: 0.8333, recall: 0.1493
2018-12-28T11:18:39.468213, step: 640, loss: 0.5360091924667358, acc: 0.6484, auc: 0.836, precision: 1.0, recall: 0.3182
2018-12-28T11:18:40.110178, step: 641, loss: 0.5488023161888123, acc: 0.7656, auc: 0.8332, precision: 0.775, recall: 0.5962
2018-12-28T11:18:40.690442, step: 642, loss: 0.632789134979248, acc: 0.5547, auc: 0.6761, precision: 0.6102, recall: 0.5143
2018-12-28T11:18:41.337429, step: 643, loss: 0.700786828994751, acc: 0.6172, auc: 0.6816, precision: 0.6027, recall: 0.6875
2018-12-28T11:18:41.977021, step: 644, loss: 0.7193906307220459, acc: 0.6484, auc: 0.6743, precision: 0.641, recall: 0.7463
2018-12-28T11:18:42.548198, step: 645, loss: 0.7353569269180298, acc: 0.5547, auc: 0.599, precision: 0.5312, recall: 0.5574
2018-12-28T1

2018-12-28T11:19:39.531201, step: 703, loss: 0.3860780894756317, acc: 0.8438, auc: 0.9061, precision: 0.9348, recall: 0.7167
2018-12-28T11:19:40.172279, step: 704, loss: 0.4370121955871582, acc: 0.8281, auc: 0.8702, precision: 0.8857, recall: 0.8158
2018-12-28T11:19:40.811673, step: 705, loss: 0.43071234226226807, acc: 0.8359, auc: 0.8873, precision: 0.8, recall: 0.8667
2018-12-28T11:19:41.412291, step: 706, loss: 0.4423547387123108, acc: 0.8281, auc: 0.8928, precision: 0.7692, recall: 0.9375
2018-12-28T11:19:41.979704, step: 707, loss: 0.4584547281265259, acc: 0.8125, auc: 0.8894, precision: 0.7424, recall: 0.875
2018-12-28T11:19:42.538911, step: 708, loss: 0.37457039952278137, acc: 0.8516, auc: 0.9138, precision: 0.8485, recall: 0.8615
2018-12-28T11:19:43.118087, step: 709, loss: 0.39698976278305054, acc: 0.8203, auc: 0.9102, precision: 0.8868, recall: 0.7344
2018-12-28T11:19:43.701479, step: 710, loss: 0.4002242088317871, acc: 0.8203, auc: 0.9001, precision: 0.8846, recall: 0.7302
2

2018-12-28T11:20:18.641933, step: 769, loss: 0.33007490634918213, acc: 0.8359, auc: 0.9527, precision: 0.9643, recall: 0.7397
2018-12-28T11:20:19.269997, step: 770, loss: 0.2771227955818176, acc: 0.8984, auc: 0.9504, precision: 0.9818, recall: 0.8182
2018-12-28T11:20:19.886326, step: 771, loss: 0.2493421733379364, acc: 0.9141, auc: 0.9577, precision: 0.9394, recall: 0.8986
2018-12-28T11:20:20.506079, step: 772, loss: 0.3210184872150421, acc: 0.875, auc: 0.9404, precision: 0.875, recall: 0.8448
2018-12-28T11:20:21.140232, step: 773, loss: 0.316522479057312, acc: 0.875, auc: 0.9398, precision: 0.9206, recall: 0.8406
2018-12-28T11:20:21.707557, step: 774, loss: 0.3606179654598236, acc: 0.8906, auc: 0.9184, precision: 0.871, recall: 0.9
2018-12-28T11:20:22.259923, step: 775, loss: 0.2316812425851822, acc: 0.9297, auc: 0.9765, precision: 0.9194, recall: 0.9344
start training model
2018-12-28T11:20:22.866418, step: 776, loss: 0.21203571557998657, acc: 0.9297, auc: 0.9721, precision: 0.9531, 

2018-12-28T11:21:19.937120, step: 833, loss: 0.1385454535484314, acc: 0.9531, auc: 0.9904, precision: 0.95, recall: 0.95
2018-12-28T11:21:20.625783, step: 834, loss: 0.3510746955871582, acc: 0.875, auc: 0.9296, precision: 0.8906, recall: 0.8636
2018-12-28T11:21:21.213545, step: 835, loss: 0.23071864247322083, acc: 0.9062, auc: 0.9658, precision: 0.9333, recall: 0.875
2018-12-28T11:21:21.796576, step: 836, loss: 0.27148932218551636, acc: 0.8828, auc: 0.9544, precision: 0.9508, recall: 0.8286
2018-12-28T11:21:22.414050, step: 837, loss: 0.2384152114391327, acc: 0.875, auc: 0.9695, precision: 0.9, recall: 0.8438
2018-12-28T11:21:23.007132, step: 838, loss: 0.3183767795562744, acc: 0.875, auc: 0.9301, precision: 0.8833, recall: 0.8548
2018-12-28T11:21:23.586066, step: 839, loss: 0.2821207046508789, acc: 0.8828, auc: 0.9555, precision: 0.9538, recall: 0.8378
2018-12-28T11:21:24.242521, step: 840, loss: 0.26925238966941833, acc: 0.8672, auc: 0.956, precision: 0.875, recall: 0.7925
2018-12-28

2018-12-28T11:21:58.932226, step: 899, loss: 0.2235698252916336, acc: 0.9219, auc: 0.9611, precision: 0.9333, recall: 0.9032
2018-12-28T11:21:59.483407, step: 900, loss: 0.19065946340560913, acc: 0.9297, auc: 0.9826, precision: 0.9483, recall: 0.9016

Evaluation:
2018-12-28T11:22:21.905910, step: 900, loss: 0.36856716086989955, acc: 0.8449947368421055, auc: 0.9262999999999999, precision: 0.8937289473684209, recall: 0.7873894736842105
2018-12-28T11:22:22.548822, step: 901, loss: 0.23797687888145447, acc: 0.9141, auc: 0.9643, precision: 0.9636, recall: 0.8548
2018-12-28T11:22:23.149891, step: 902, loss: 0.2272333800792694, acc: 0.9141, auc: 0.9662, precision: 0.9701, recall: 0.8784
2018-12-28T11:22:23.740817, step: 903, loss: 0.23763276636600494, acc: 0.9141, auc: 0.9651, precision: 0.918, recall: 0.9032
2018-12-28T11:22:24.307419, step: 904, loss: 0.33594149351119995, acc: 0.8516, auc: 0.9299, precision: 0.8929, recall: 0.7937
2018-12-28T11:22:24.879804, step: 905, loss: 0.2471592426300